<a href="https://colab.research.google.com/github/fernandolievano/ProyectoFinal_PP1/blob/entrega-final%2Fparcial-2/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 1. Introducción

In [1]:
# @title
from urllib.request import urlopen
from IPython.display import HTML

RAW_URL = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/assets/doc.html"
with urlopen(RAW_URL) as f:
    html = f.read().decode("utf-8", errors="replace")
HTML(html)

# 2. Configuración del entorno

### 2.1 Instalación de Librerías

In [2]:
%pip install pandas numpy plotly

### 2.2 Importación de Módulos

In [3]:
import pandas as pd
import plotly.express as px

### 2.3 Carga del Dataset

In [5]:
dataset_url = "https://raw.githubusercontent.com/fernandolievano/ProyectoFinal_PP1/main/data/stack-overflow-developer-survey-2025.csv.gz"
df = pd.read_csv(dataset_url, compression='gzip')

print("✅ DataSet cargado con éxto")

✅ DataSet cargado con éxto


### 2.4 Funciones y Variables Utilitarias

In [6]:
# Funciones Reutilizables
def contar_valores_explodidos(dataframe, columna, separador):
    """
    Cuenta las incidencias de valores en una columna que contiene strings
    separados por un delimitador.
    :param dataframe:
    :param columna:
    :param separador:
    :return:
        Una Serie de pandas
    """
    if columna not in dataframe.columns:
        raise ValueError(f"Columna {columna} no encontrada")

    return (dataframe[columna]
            .str.split(separador)
            .explode()  # cada elemento de la lista se vuelve una fila
            .value_counts())


# Variables Reutilizables
n_encuestados = len(df)

# 3. Exploración Inicial del Dataset

### 3.1 Estructura y Dimensiones

In [7]:
print(f"Dimensiones del Dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

Dimensiones del Dataset: (49123, 170)
Número de filas: 49123
Número de columnas: 170


### 3.2 Información general del Dataset

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49123 entries, 0 to 49122
Columns: 170 entries, ResponseId to JobSat
dtypes: float64(50), int64(1), object(119)
memory usage: 63.7+ MB


### 3.3 Head y Tail para muestras

In [9]:
df.head()

,ResponseId,MainBranch,Age,EdLevel,Employment,EmploymentAddl,WorkExp,LearnCodeChoose,LearnCode,LearnCodeAI,...,AIAgentOrchestration,AIAgentOrchWrite,AIAgentObserveSecure,AIAgentObsWrite,AIAgentExternal,AIAgentExtWrite,AIHuman,AIOpen,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,25-34 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Employed,"Caring for dependents (children, elderly, etc.)",8.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,Vertex AI,NaN,NaN,NaN,ChatGPT,NaN,When I don’t trust AI’s answers,"Troubleshooting, profiling, debugging",61256.0,10.0
1,2,I am a developer by profession,25-34 years old,"Associate degree (A.A., A.S., etc.)",Employed,NaN,2.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers;When I want to...,All skills. AI is a flop.,104413.0,9.0
2,3,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Independent contractor, freelancer, or self-em...",None of the above,10.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code;GitHub Copilot;Google Gemini,NaN,When I don’t trust AI’s answers;When I want to...,"Understand how things actually work, problem s...",53061.0,8.0
3,4,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,None of the above,4.0,"Yes, I am not new to coding but am learning ne...","Other online resources (e.g. standard search, ...","Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code,NaN,When I don’t trust AI’s answers;When I want to...,NaN,36197.0,6.0
4,5,I am a developer by profession,35-44 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...","Caring for dependents (children, elderly, etc.)",21.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers,"critical thinking, the skill to define the tas...",60000.0,7.0


In [10]:
df.tail()

,ResponseId,MainBranch,Age,EdLevel,Employment,EmploymentAddl,WorkExp,LearnCodeChoose,LearnCode,LearnCodeAI,...,AIAgentOrchestration,AIAgentOrchWrite,AIAgentObserveSecure,AIAgentObsWrite,AIAgentExternal,AIAgentExtWrite,AIHuman,AIOpen,ConvertedCompYearly,JobSat
49118,49119,I am a developer by profession,25-34 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,NaN,9.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools req...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
49119,49120,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,"Caring for dependents (children, elderly, etc.)",13.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools req...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49120,49121,I am a developer by profession,25-34 years old,"Secondary school (e.g. American high school, G...",Employed,NaN,2.0,"Yes, I am not new to coding but am learning ne...","Other online resources (e.g. standard search, ...","No, I didn't spend time learning in the past year",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49121,49122,I am a developer by profession,25-34 years old,"Associate degree (A.A., A.S., etc.)",Employed,None of the above;Engaged in paid work (20-29 ...,10.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
49122,49123,I am a developer by profession,25-34 years old,Some college/university study without earning ...,Employed,None of the above,10.0,"No, I am not new to coding and did not learn n...",NaN,"No, I learned something that was not related t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0


### 3.4 Estadísticas descriptivas

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ResponseId,49123.0,2.456200e+04,1.418073e+04,1.0,12281.5,24562.0,36842.50,4.912300e+04
WorkExp,42844.0,1.337081e+01,1.080192e+01,1.0,5.0,10.0,20.00,1.000000e+02
YearsCode,43000.0,1.657526e+01,1.178927e+01,1.0,8.0,14.0,24.00,1.000000e+02
TechEndorse_1,35943.0,7.867763e+00,2.397243e+00,1.0,7.0,9.0,9.00,1.400000e+01
TechEndorse_2,35943.0,4.104415e+00,2.276154e+00,1.0,2.0,4.0,6.00,1.400000e+01
TechEndorse_3,35943.0,4.110341e+00,2.329539e+00,1.0,2.0,4.0,6.00,1.400000e+01
TechEndorse_4,35943.0,5.678574e+00,2.397899e+00,1.0,4.0,6.0,8.00,1.400000e+01
TechEndorse_5,35943.0,4.119383e+00,2.437938e+00,1.0,2.0,4.0,6.00,1.400000e+01
TechEndorse_6,35943.0,5.225830e+00,2.801354e+00,1.0,3.0,5.0,8.00,1.400000e+01
TechEndorse_7,35943.0,6.477562e+00,2.331433e+00,1.0,5.0,7.0,8.00,1.400000e+01


# 4. Análisis Exploratorio de Datos (EDA)
Llevaremos a cabo un análisis exploratorio de datos utilizando visualizaciones interactivas y estadísticas descriptivas, con el objetivo de comprender mejor los resultados de la encuesta hecha por Stack Overflow en 2025. Mediante este proceso podremos responder preguntas sobre lenguajes de programación más usados, tecnologías en tendencias, roles comunes y niveles salariales.

## Verificación de nulos

In [ ]:
pd.set_option('display.max_rows', None)  # Activo display.max_rows para poder ver todos mis columnas nulas

tiene_nulos = df.isnull().sum() > 0  # Máscara para mostrar solo resultados con valores nulos
nulos = df.isnull().sum()[tiene_nulos]
nulos_cantidad = len(nulos)
nulos_porcentaje = (nulos / n_encuestados) * 100

print(f"❗ Encontramos {nulos_cantidad} columnas con datos nulos")
print("ℹ️ Porcentaje de valores nulos encontrados por columna:")

print(nulos_porcentaje.sort_values(ascending=False).to_frame(name="Porcentaje"))

pd.reset_option('display.max_rows', None)  # Desactivo de nuevo display.max_rows

### Altos nulos
En este gráfico se muestran las variables con mayor porcentaje de valores faltantes.
Dado que superan ampliamente el 80% de nulos, pueden considerarse prescindibles para el análisis, ya que su nivel de completitud es insuficiente para obtener insights confiables.

In [ ]:
sobre_80_porciento = nulos_porcentaje > 80  # Máscara obtener resultados mayores a 80%
nulos_altos = nulos_porcentaje[sobre_80_porciento].sort_values(ascending=False)

# Gráfico
fig_altos_nulos = px.bar(
    nulos_altos,
    color='value',
    orientation='h',
    title='⚠️ Variables con más del 80% de valores nulos',
    labels={'value': 'Porcentaje de Nulos', 'index': 'Variable'},
    color_continuous_scale='Reds'
)
fig_altos_nulos.update_layout(
    height=40 * len(nulos_altos),
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_altos_nulos.show()

### Nulos en variables clave
Se muestran variables fundamentales del análisis (como país, educación y años de experiencia) que presentan un bajo porcentaje de valores faltantes, lo que permite tratarlos sin afectar la calidad de los resultados.

In [ ]:
variables_clave = ['Country', 'Employment', 'EdLevel', 'WorkExp', 'YearsCode', 'ConvertedCompYearly']
nulos_claves = nulos[variables_clave].sort_values(ascending=False)

# Gráfico
fig_nulos_clave = px.bar(
    nulos_claves,
    title='🔑 Cantidad de nulos en variables clave',
    labels={'value': 'Cantidad de Nulos', 'index': 'Variable'},
    color='value',
    color_continuous_scale='sunsetdark'
)
fig_nulos_clave.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    title=dict(
        xanchor='center',
        x=0.5
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
)
fig_nulos_clave.show()

 # Análisis de hipótesis
 En esta sección se abordan las preguntas planteadas al inicio del proyecto, utilizando gráficos interactivos y resúmenes numéricos para explorarlas en detalle.

## 1. ¿Qué lenguajes de programación están en auge?

In [ ]:
# Usamos la función "contar_valores_explodidos" para dividir los valores
# de la columna 'LanguageHaveWorkedWith' (separados por ';') y obtener
# un conteo de los lenguajes más utilizados.
lenguajes_usados_exploded = contar_valores_explodidos(dataframe=df, columna='LanguageHaveWorkedWith', separador=';')
lenguajes_usados_exploded = lenguajes_usados_exploded.sort_values(ascending=True)

fig_lenguajes_usados = px.bar(
    lenguajes_usados_exploded,
    title='👨‍💻 Lenguajes de programación más utilizados en 2025',
    color='value',
    color_continuous_scale='pinkyl',
    orientation='h',
    labels={'value': 'Uso', 'LanguageHaveWorkedWith': 'Lenguaje de Programación'},
)
fig_lenguajes_usados.update_layout(
    font=dict(
        size=14,
        color='#fff',
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    height=36 * len(lenguajes_usados_exploded),
    title=dict(
        xanchor='center',
        x=0.5
    ),
)

fig_lenguajes_usados.show()

> 🔍 **El ecosistema web domina el uso**. JavaScript, HTML/CSS y SQL lideran con Python muy cerca. Además, TypeScript se posiciona por encima de Java/C#, evidenciando la consolidación del front-end “profesional”.

In [ ]:
# === Expectativas de Lenguajes (auge) ===
# Gap (pp) = % que desea usar − % que usa hoy, ambos sobre el total de encuestados.

# 1) Conteos de "deseo"
lenguajes_deseados = contar_valores_explodidos(
    dataframe=df,
    columna='LanguageWantToWorkWith',
    separador=';'
)

# 2) Normalización a % del total de encuestados
lenguajes_deseados_pct = (lenguajes_deseados / n_encuestados) * 100
lenguajes_usados_pct   = (lenguajes_usados_exploded / n_encuestados) * 100

# 3) Alineación por índice (si un lenguaje aparece en una lista y no en la otra, tomar 0)
todos = lenguajes_deseados_pct.index.union(lenguajes_usados_pct.index)
deseo_pp = lenguajes_deseados_pct.reindex(todos, fill_value=0.0)
uso_pp   = lenguajes_usados_pct.reindex(todos, fill_value=0.0)

# 4) Gap de auge (pp) y filtro de positivos
lenguajes_auge = (deseo_pp - uso_pp)
lenguajes_auge = lenguajes_auge[lenguajes_auge > 0].sort_values(ascending=True)

# 5) Gráfico
import plotly.express as px

fig_lenguajes_auge = px.bar(
    lenguajes_auge,
    title="📈 Lenguajes de Programación en Auge en 2025",
    color="value",
    color_continuous_scale="tempo",
    orientation="h",
    labels={"index": "Lenguajes de Programación", "value": "% auge (pp)"},
)

# Escala de color consistente (desde 0 hasta el máximo) y título del colorbar
fig_lenguajes_auge.update_coloraxes(
    cmin=0,
    cmax=float(lenguajes_auge.max()),
    colorbar_title="% auge (pp)",
)

# Hover claro
fig_lenguajes_auge.update_traces(
    hovertemplate="Lenguaje: %{y}<br>Auge: %{x:.2f} pp<extra></extra>"
)

# Estilo general y altura con tope
fig_lenguajes_auge.update_layout(
    font=dict(size=14, color="#fff"),
    paper_bgcolor="black",
    plot_bgcolor="black",
    margin=dict(l=220, r=40, t=80, b=40),
    height=min(36 * len(lenguajes_auge), 1200),
    title=dict(
        xanchor="center",
        x=0.5,
        text=(
            "📈 Lenguajes de Programación en Auge en 2025"
            f"<br><sup>Gap = deseo − uso (pp) sobre el total de encuestados, n = {n_encuestados}</sup>"
        ),
    ),
)

fig_lenguajes_auge.show()


> 🧠 El gráfico muestra los lenguajes con gap positivo (deseo − uso, en puntos porcentuales sobre el total de encuestados), es decir, aquellos que hoy son más deseados que utilizados. Este patrón sugiere potencial de adopción en el corto/mediano plazo: lenguajes emergentes tienden a liderar por interés futuro elevado pese a una base actual pequeña. En términos prácticos, estos resultados apuntan a oportunidades de formación y a una posible reorientación de stack hacia tecnologías con mayor tracción aspiracional.

***Nota***: la lectura debe considerar el tamaño de muestra por lenguaje (los de base reducida son más volátiles) y que este indicador refleja intención declarada, no adopción efectiva.

## 2. ¿Qué tecnologías muestran menor interés futuro?

In [ ]:
# === Tecnologías que pierden interés (gap negativo) ===
# Gap (pp) = % que desea usar − % que usa hoy, ambos sobre el total de encuestados.
lenguajes_retroceso = (lenguajes_deseados_pct - lenguajes_usados_pct)

# Filtramos sólo retroceso (< 0) y ordenamos de más negativo (mayor retroceso) a menos negativo.
lenguajes_retroceso = lenguajes_retroceso[lenguajes_retroceso < 0].sort_values(ascending=True)

# Gráfico de barras horizontal (Series -> usa 'index' y 'value')
import plotly.express as px

fig_lenguajes_retroceso = px.bar(
    lenguajes_retroceso,
    title="📉 Tecnologías que pierden interés en 2025",
    color='value',                       # escala por magnitud del retroceso
    color_continuous_scale='pinkyl_r',
    orientation='h',
    labels={'index': 'Lenguajes de Programación', 'value': '% retroceso (pp)'}
)

# Escala de color consistente (máximo en 0) y título del colorbar
fig_lenguajes_retroceso.update_coloraxes(
    cmin=lenguajes_retroceso.min(),
    cmax=0,
    colorbar_title="% retroceso (pp)"
)

# Hover claro
fig_lenguajes_retroceso.update_traces(
    hovertemplate="Lenguaje: %{y}<br>Retroceso: %{x:.2f} pp<extra></extra>"
)

# Estilo general
fig_lenguajes_retroceso.update_layout(
    font=dict(size=14, color="#fff"),
    paper_bgcolor="black",
    plot_bgcolor="black",
    margin=dict(l=220, r=40, t=80, b=40),
    height=min(36 * len(lenguajes_retroceso), 1200),
    title=dict(
        xanchor="center",
        x=0.5,
        text=f"📉 Tecnologías que pierden interés en 2025"
             f"<br><sup>Gap = deseo − uso (pp) sobre el total de encuestados, n = {n_encuestados}</sup>"
    ),
)

fig_lenguajes_retroceso.show()


> 🧠 El gráfico presenta las tecnologías con gap negativo. La diferencia entre quienes desean usarlas y quienes las usan hoy, en puntos porcentuales sobre el total de encuestados, lo que indica desinterés relativo o pérdida de relevancia a futuro. En varias tecnologías masivas (p. ej., JS/HTML-CSS/SQL) el retroceso sugiere saturación/estabilidad más que abandono: mantienen alta adopción, pero una proporción menor planea seguir eligiéndolas. Estos resultados invitan a diversificar habilidades hacia opciones con mayor tracción aspiracional, sin sobrerreaccionar: la lectura debe considerar el tamaño de muestra por lenguaje y que la métrica refleja intención declarada, no adopción efectiva.